In [1]:
import sys

PWD = '../../..'
sys.path.append(PWD)

import numpy as np
import pandas as pd

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params
from src.cross_validator import KerasTunerAllChannelsCrossValidator
from src.models import convnet_builder as bare_model_builder
from src.dataset import load_dataset_train_val_all_channels, \
    load_dataset_train_test_all_channels

In [2]:
PROJECT_NAME = 'convnet'

OVERWRITE = False

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_all/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_all/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 500
BATCH_SIZE = 8096
MAX_TRIALS = 40
EXECUTIONS_PER_TRIAL = 2

TOP_N = 5
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_all, _, y_all, _ = load_dataset_train_test_all_channels(PWD)

X_all[(2, 11)].shape, y_all[(2, 11)].shape

((22412, 24), (22412,))

In [4]:
X_train, X_val, y_train, y_val = load_dataset_train_val_all_channels(PWD)

X_train = np.concatenate(list(X_train.values()))
y_train = np.concatenate(list(y_train.values()))
X_val = np.concatenate(list(X_val.values()))
y_val = np.concatenate(list(y_val.values()))

rng = np.random.default_rng(seed=42)

train_idx, val_idx = np.arange(len(X_train)), np.arange(len(X_val))
rng.shuffle(train_idx)
rng.shuffle(val_idx)

X_train, y_train = X_train[train_idx], y_train[train_idx]
X_val, y_val = X_val[val_idx], y_val[val_idx]

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((85651, 24), (85651,), (21416, 24), (21416,))

# Model

In [5]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    # Convolutional network params
    hp_n_conv_blocks = hp.Int("n_conv_blocks", min_value=1, max_value=4, step=1)
    hp_n_conv_layers = hp.Int("n_conv_layers", min_value=1, max_value=3, step=1)
    hp_filters_mult = hp.Choice("conv_filters_mult", values=[1, 2, 4, 8])
    hp_conv_spatial_dropout = hp.Choice("conv_spatial_dropout", values=[0.0, 0.1, 0.2])

    # MLP at the end params
    hp_mlp_n_hidden_layers = hp.Int("n_mlp_hidden_layers", min_value=0, max_value=3, step=1, default=0)
    hp_mlp_units_mult, hp_mlp_dropout = None, None
    if hp_mlp_n_hidden_layers > 0:
        hp_mlp_units_mult = hp.Choice("mlp_units_mult", values=[1, 2, 4, 8, 16], default=4)
        hp_mlp_dropout = hp.Choice("mlp_dropout", values=[0.0, 0.2, 0.5])

    # Other params
    hp_batch_normalization = hp.Boolean("batch_normalization", default=False)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization", default=False)
    hp_normalize_signal = hp.Boolean("normalize_signal", default=False)

    model = bare_model_builder(hp_n_conv_blocks, hp_n_conv_layers, hp_filters_mult, hp_conv_spatial_dropout,
                               hp_mlp_n_hidden_layers, hp_mlp_units_mult, hp_mlp_dropout, hp_batch_normalization,
                               hp_input_batch_normalization, hp_normalize_signal)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [6]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 24, 1)             0         
                                                                 
 conv1d (Conv1D)             (None, 24, 8)             32        
                                                                 
 flatten (Flatten)           (None, 192)               0         
                                                                 
 dense (Dense)               (None, 1)                 193       
                                                                 
Total params: 225
Trainable params: 225
Non-trainable params: 0
_________________________________________________________________


In [7]:
model_callbacks = [
    callbacks.EarlyStopping(patience=30, min_delta=ES_MIN_DELTA),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=5)
]

# Bayesian tuner

In [8]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

# Results

In [9]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'n_conv_blocks': 3, 'n_conv_layers': 3, 'conv_filters_mult': 8, 'conv_spatial_dropout': 0.1, 'n_mlp_hidden_layers': 3, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 4, 'mlp_dropout': 0.0}
Number of parameters: 747425
========== Model 1 ==========
{'n_conv_blocks': 4, 'n_conv_layers': 1, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 1, 'batch_normalization': False, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 8, 'mlp_dropout': 0.0}
Number of parameters: 44993
========== Model 2 ==========
{'n_conv_blocks': 3, 'n_conv_layers': 2, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 0, 'batch_normalization': False, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 4, 'mlp_dropout': 0.2}
Number of parameters: 24561
========== Model 3 ==========
{'n_conv_blocks': 2, 'n_conv_layers':

# Cross-validation for top 5 models

In [10]:
cross_validator = KerasTunerAllChannelsCrossValidator(bayesian_tuner, list(X_all.values()), list(y_all.values()),
                                                      model_builder, directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                                      n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                                      es_min_delta=ES_MIN_DELTA, n_cv=CROSSVAL_N_CV,
                                                      n_executions=CROSSVAL_N_EXEC, overwrite=OVERWRITE)
model_scores = cross_validator()

{'n_conv_blocks': 3, 'n_conv_layers': 3, 'conv_filters_mult': 8, 'conv_spatial_dropout': 0.1, 'n_mlp_hidden_layers': 3, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 4, 'mlp_dropout': 0.0}
Number of parameters: 747425
Got score: 8.2443 (8.3798, 8.1088)
Got score: 8.6424 (8.5794, 8.7055)
Got score: 8.2703 (8.1736, 8.3671)
Got score: 8.2887 (8.2149, 8.3624)
Got score: 8.3834 (8.5026, 8.2642)


{'n_conv_blocks': 4, 'n_conv_layers': 1, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 1, 'batch_normalization': False, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 8, 'mlp_dropout': 0.0}
Number of parameters: 44993
Got score: 8.4172 (8.4617, 8.3727)
Got score: 8.6656 (8.9292, 8.4021)
Got score: 8.6966 (8.6649, 8.7283)
Got score: 8.4951 (8.3846, 8.6057)
Got score: 8.2889 (8.2995, 8.2782)


{'n_conv_blocks': 3, 'n_conv_layers': 2, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 0, 'batch_normalization': False, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 4, 'mlp_dropout': 0.2}
Number of parameters: 24561
Got score: 8.4984 (8.5389, 8.4578)
Got score: 8.5531 (8.5048, 8.6015)
Got score: 8.5032 (8.4517, 8.5547)
Got score: 8.4749 (8.4975, 8.4522)
Got score: 8.3008 (8.2863, 8.3152)


{'n_conv_blocks': 2, 'n_conv_layers': 3, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.2, 'n_mlp_hidden_layers': 1, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 4, 'mlp_dropout': 0.0}
Number of parameters: 50881
Got score: 8.3839 (8.3787, 8.3892)
Got score: 8.7178 (8.5809, 8.8546)
Got score: 8.4452 (8.4858, 8.4046)
Got score: 8.5906 (8.5478, 8.6333)
Got score: 8.2253 (8.1923, 8.2583)


{'n_conv_blocks': 4, 'n_conv_layers': 1, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 0, 'batch_normalization': False, 'input_batch_normalization': True, 'normalize_signal': True, 'mlp_units_mult': 16, 'mlp_dropout': 0.5}
Number of parameters: 33025
Got score: 8.8552 (8.8659, 8.8444)
Got score: 8.7075 (8.6658, 8.7493)
Got score: 8.7730 (8.7557, 8.7902)
Got score: 8.9186 (9.1974, 8.6399)
Got score: 8.6795 (8.6652, 8.6938)


In [11]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,8.37,0.15,747425
1,8.51,0.15,44993
2,8.47,0.09,24561
3,8.47,0.17,50881
4,8.79,0.09,33025
